# Holt-Winters

In [1]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# Load the dataset
df = pd.read_csv('data/interpolated_co2.csv')

In [3]:
# Convert 'Date' to datetime format and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [4]:
# Define training and testing periods
train_end_date = '2024-12-31'
test_start_date = '2025-01-01'
test_end_date = '2025-04-13'

In [5]:
# Split the data
train = df.loc[:train_end_date]
test = df.loc[test_start_date:test_end_date]

In [6]:
# Fit the Holt-Winters model (additive trend and seasonality with period 365)
model = ExponentialSmoothing(
    train['CO2'],
    seasonal='add',
    trend='add',
    seasonal_periods=365
)
fitted_model = model.fit()

/Users/lucifer/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/lucifer/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [7]:
# Forecast for the test period
predictions = fitted_model.forecast(len(test))

In [8]:
# Evaluate predictions
rmse = np.sqrt(mean_squared_error(test['CO2'], predictions))
mae = mean_absolute_error(test['CO2'], predictions)
accuracy = 100 - (mae / test['CO2'].mean() * 100)

In [9]:
# Print evaluation metrics
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"Accuracy: {accuracy:.2f}%")

RMSE: 1.1079
MAE: 0.9234
Accuracy: 99.78%


In [10]:
# Create DataFrame for predictions
predictions_df = pd.DataFrame({
    'Date': test.index,
    'Actual': test['CO2'].values,
    'Predicted': predictions.values
})
predictions_df.set_index('Date', inplace=True)

# Save predictions to a CSV file in 'Predictions' folder
output_dir = 'Predictions'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'holt_winters_predictions.csv')
predictions_df.to_csv(output_path)